In [ ]:
import sys
import networkx as nx
import numpy as np

In [ ]:
sys.path.append("../")

## Sanity check

In [ ]:
from qtensor import QAOA_energy

G = nx.random_regular_graph(3, 10)
gamma, beta = [np.pi/3], [np.pi/2]

E = QAOA_energy(G, gamma, beta)

## Vertex Cover: Qiskit

In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
# Qtensor branch: dev
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from qtensor import QAOAComposer
from qtensor import CirqQAOAComposer, QtreeQAOAComposer
from qtensor import VCQiskitQAOAComposer, PCQiskitQAOAComposer
from qtensor import VCQtreeQAOAComposer, PCQtreeQAOAComposer

### QAOA problem graph

In [ ]:

G = nx.erdos_renyi_graph(4, 2/(5-1))
nx.draw_kamada_kawai(G, with_labels=True)

In [ ]:
p = 1
qiskit_qaoa = VCQiskitQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
qiskit_qaoa.ansatz_state()
qiskit_qaoa.circuit.draw('mpl')

In [ ]:
p = 1
qiskit_qaoa = PCQiskitQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
qiskit_qaoa.ansatz_state()
qiskit_qaoa.circuit.draw('mpl')

## Simulate circuits
## Use lightcone optimisaiton

Suppose we are interested in an expectation value of particular operator in a state $|\psi\rangle = \hat U | 0\rangle$. 
We can use the fact that in the expression
$$\langle \psi | \hat E | \psi \rangle = \langle 0 | \hat U^\dagger \hat E \hat U |0\rangle$$
a lot of operators from $\hat U$ cancel out.

In [ ]:
p = 2
qiskit_qaoa = VCQiskitQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
qiskit_qaoa.energy_expectation_lightcone((0,1))
qiskit_qaoa.circuit.draw('mpl')

In [ ]:
p = 2
qiskit_qaoa = PCQiskitQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
qiskit_qaoa.energy_expectation_lightcone((0,1))
qiskit_qaoa.circuit.draw('mpl')

In [ ]:
com = VCQtreeQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
com.energy_expectation_lightcone(list(G.edges())[0])

In [ ]:
com = PCQtreeQAOAComposer(G, gamma=[.1]*p, beta=[.2]*p)
com.energy_expectation_lightcone(list(G.edges())[0])

In [ ]:
from qtensor.QAOASimulator import QAOAQtreeSimulator
qaoa_sim = QAOAQtreeSimulator(VCQtreeQAOAComposer)

qaoa_sim.energy_expectation(G, gamma=[.1]*p, beta=[.2]*p)

In [ ]:
from qtensor.QAOASimulator import QAOAQtreeSimulator
qaoa_sim = QAOAQtreeSimulator(PCQtreeQAOAComposer)

p=3
qaoa_sim.energy_expectation(G, gamma=[.1]*p, beta=[.2]*p)

## QAOA

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
graph = nx.Graph(edges)

nx.draw(graph, with_labels=True)
plt.show()

In [ ]:

p=3
initial_params = np.array([.1]*2*p)

In [ ]:
def cost_function(params):
    p = len(params) // 2

    # Split the single list into two lists
    gammas = params[:p]
    betas = params[p:]
    expectation = qaoa_sim.energy_expectation(graph, gammas, betas)
    return expectation

In [ ]:
# Minimize the function
result = minimize(cost_function, initial_params)

In [ ]:
result